In [8]:
%%bigquery category_df
select c0_name, c1_name, c2_name from `solutions-2023-mar-107.mercari.13K_synthetic_attributes_embeddings`
group by c0_name, c1_name, c2_name

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
%%bigquery df
select b.*, a.manual_validation from 
`solutions-2023-mar-107.mercari.13K_synthetic_attributes_embeddings_validation` a
JOIN 
`solutions-2023-mar-107.mercari.13K_synthetic_attributes_embeddings` b
ON a.id = b.id

Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
df.head(3)

,id,name,description,brand_name,item_condition_name,c0_name,c1_name,c2_name,url,created,image_uri,vision_api_labels,attributes,scores,text_embedding,image_embedding,manual_validation
0,m47655598835,Violator Action Figure 1994 Loose Spawn First ...,Violator Action Figure 1994 Loose Spawn First ...,McFarlane Toys,Good,Toys & Collectibles,Action Figures & Accessories,None,https://www.mercari.com/us/item/m47655598835,2023-03-12 06:49:22+00:00,gs://genai-product-catalog/mercari_images_13K/...,"{""label_annotations"":[{""description"":""Art"",""mi...","[Art, Fashion design, Costume design, Visual a...","[0.8215251, 0.77831143, 0.67183554, 0.67093885...","[0.0360803641, -0.0278392285, 0.00694904197, 0...","[0.00795026217, 0.00359571818, 0.0397700928, -...",0
1,m96082559260,Dr Teeth Rowlf Figure Muppets Minimates Series...,"The show must go on! The most sensational, ins...",Diamond Select,New,Toys & Collectibles,Action Figures & Accessories,Action Figures,https://www.mercari.com/us/item/m96082559260,2023-03-16 12:12:47+00:00,gs://genai-product-catalog/mercari_images_13K/...,"{""label_annotations"":[{""description"":""Toy"",""mi...","[Toy, Font, Goggles, Fictional character, Plas...","[0.87547928, 0.76024967, 0.70768821, 0.6879748...","[0.00385425612, -0.0133601641, 0.0112476563, -...","[-0.0547101572, -0.00857442431, 0.0384701677, ...",0
2,m14193490298,Wooden Magnet/ Dot Art Acrylic Paint,Handfree painted mandalas in a thin wooden dis...,Handmade,New,Home,Artwork,Paintings,https://www.mercari.com/us/item/m14193490298,2023-02-06 04:48:31+00:00,gs://genai-product-catalog/mercari_images_13K/...,"{""label_annotations"":[{""description"":""Book"",""m...","[Book, Art, Publication, Creative arts, Materi...","[0.87862146, 0.83368653, 0.81959659, 0.8156618...","[0.00826694351, 0.00405315682, 0.0570168681, -...","[0.0284959301, 0.0240893103, 0.017579874, -0.0...",1


In [11]:
def get_category_name(row):
    if row["c2_name"] is None:
        cat_name = row["c0_name"]+">"+row["c1_name"]+">"+"Other"
    else:
        cat_name = row["c0_name"]+">"+row["c1_name"]+">"+row["c2_name"]
    return cat_name

df["category"] = df.apply(lambda x: get_category_name(x), axis=1)

In [12]:
df["comb_embedding"] = df.apply(lambda x: x["text_embedding"].tolist()+x["image_embedding"].tolist(), axis=1)

In [28]:
import joblib # (if needed)
model = joblib.load('model_textemb.pkl')
embeddings = "text_embedding"

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

df["predicted_category"] = model.predict(df[embeddings].tolist())

df_val = df[df["manual_validation"]==1]

print("Accuracy: " + str(accuracy_score(df_val["predicted_category"], df_val["category"])))
print("f1: " + str(f1_score(df_val["category"], df_val["predicted_category"],average='weighted')))

Accuracy: 0.4224137931034483
f1: 0.34441614420062694


In [30]:
# hierarchical classification

def get_similarity_score(y_true, y_pred):
    
    score = 0
    
    y_true_ls = y_true.split(">")
    y_pred_ls = y_pred.split(">")
    if len(y_true_ls) != 3 or len(y_pred_ls) != 3:
        print("error: category does not have 3 levels")
        return "nada"
    
    if y_true_ls[0] == y_pred_ls[0]:
        score += 1/3
    if y_true_ls[1] == y_pred_ls[1]:
        score += 1/3
    if y_true_ls[2] == y_pred_ls[2]:
        score += 1/3
    
    return score

df["score"] = df.apply(lambda x: get_similarity_score(x["category"],x["predicted_category"]), axis=1)

df[df["manual_validation"]==1]["score"].mean()

0.6206896551724137

In [24]:
import pandas as pd
pd.set_option('display.max_rows', None)
df[df["manual_validation"]==1]

,id,name,description,brand_name,item_condition_name,c0_name,c1_name,c2_name,url,created,...,vision_api_labels,attributes,scores,text_embedding,image_embedding,manual_validation,category,comb_embedding,predicted_category,score
2,m14193490298,Wooden Magnet/ Dot Art Acrylic Paint,Handfree painted mandalas in a thin wooden dis...,Handmade,New,Home,Artwork,Paintings,https://www.mercari.com/us/item/m14193490298,2023-02-06 04:48:31+00:00,...,"{""label_annotations"":[{""description"":""Book"",""m...","[Book, Art, Publication, Creative arts, Materi...","[0.87862146, 0.83368653, 0.81959659, 0.8156618...","[0.00826694351, 0.00405315682, 0.0570168681, -...","[0.0284959301, 0.0240893103, 0.017579874, -0.0...",1,Home>Artwork>Paintings,"[0.00826694351, 0.00405315682, 0.0570168681, -...",Home>Home decor>Home decor accents,0.333333
3,m74667116621,POSTER PRINT: FUNHOUSE,ALL POSTER PRINTS ARE 11 X 17 INCHES (( GREAT ...,None,New,Home,Artwork,Posters,https://www.mercari.com/us/item/m74667116621,2023-01-25 07:01:14+00:00,...,"{""label_annotations"":[{""description"":""Poster"",...","[Poster, Publication, Font, Art, Book cover, B...","[0.86772853, 0.83406168, 0.81918359, 0.6839227...","[-0.0161228031, -0.0427463576, -0.0123792794, ...","[-0.0586416498, 0.0408442616, -0.012638161, -0...",1,Home>Artwork>Posters,"[-0.0161228031, -0.0427463576, -0.0123792794, ...",Books>Fiction Books>Comics,0.000000
4,m21554068673,Athleta Elation Purple Velvet High Rise Tight ...,Athleta Elation Purple Blue Velvet Tight Leggi...,Athleta,Like new,Women,Athletic apparel,Athletic Leggings,https://www.mercari.com/us/item/m21554068673,2023-03-27 17:33:50+00:00,...,"{""label_annotations"":[{""description"":""Arm"",""mi...","[Arm, Shoulder, yoga pant, Leg, Active pants, ...","[0.94758928, 0.93993074, 0.93505448, 0.9231663...","[0.0108732795, -0.0630845651, 0.036399167, -0....","[0.0493304357, -0.0114150364, 0.0233411063, 0....",1,Women>Athletic apparel>Athletic Leggings,"[0.0108732795, -0.0630845651, 0.036399167, -0....",Women>Athletic apparel>Athletic Leggings,1.000000
5,m67375259941,Mens FootJoy 1/4 Zip Tonal Heather Golf Pullov...,Size 2Xl\nMade in Vietnam\n100% Polyester\nTon...,FootJoy,Good,Men,Athletic apparel,Athletic Sweatshirts,https://www.mercari.com/us/item/m67375259941,2023-03-12 17:54:11+00:00,...,"{""label_annotations"":[{""description"":""Sleeve"",...","[Sleeve, Grey, T-shirt, Collar, Active shirt, ...","[0.87245047, 0.84274274, 0.78312498, 0.7625238...","[0.0175424833, -0.0356829725, 0.0442052484, 0....","[0.00461410824, 0.00984642282, 0.0379393771, -...",1,Men>Athletic apparel>Athletic Sweatshirts,"[0.0175424833, -0.0356829725, 0.0442052484, 0....","Men>Sweats & hoodies>Sweatshirt, pullover",0.333333
6,m67616241887,Nike Jordan Georgetown Jersey,Nike Jordan College Dri-FIT Spotlight Georgeto...,Jordan,Like new,Men,Athletic apparel,Jerseys,https://www.mercari.com/us/item/m67616241887,2023-03-25 01:42:31+00:00,...,"{""label_annotations"":[{""description"":""Outerwea...","[Outerwear, White, Baby & toddler clothing, Sl...","[0.95222831, 0.92185944, 0.87331194, 0.8724504...","[0.00263984082, -0.0399066284, 0.0268216655, 0...","[0.0359478854, 0.0247595552, 0.0239854828, -0....",1,Men>Athletic apparel>Jerseys,"[0.00263984082, -0.0399066284, 0.0268216655, 0...",Men>Tops>T-shirts,0.333333
7,m14055943821,Nike Sculpt hyper leggings,NWT\nSize small\nhigh rise,Nike,New,Women,Athletic apparel,Athletic Leggings,https://www.mercari.com/us/item/m14055943821,2023-04-01 02:51:20+00:00,...,"{""label_annotations"":[{""description"":""Shorts"",...","[Shorts, Sleeve, Sportswear, Electric blue, T-...","[0.94813758, 0.87245035, 0.72904754, 0.7257895...","[0.0172865782, -0.0055040326, 0.0136040561, 0....","[0.0581621043, -0.0179830398, 0.0268484969, 0....",1,Women>Athletic apparel>Athletic Leggings,"[0.0172865782, -0.0055040326, 0.0136040561, 0....",Women>Athletic apparel>Athletic Leggings,1.000000
8,m98238658538,NWT Lululemon Matching Set - Size 6 in 'Blue B...,BRAND NEW WITH TAGS☺️\n